<a href="https://colab.research.google.com/github/RaedHadad/Elephant/blob/main/Elephant_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install reportlab
!pip install matplotlib
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.5 MB/s eta 0:00:00


In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import requests
import json
import nltk
from nltk.chat.util import Chat, reflections
from collections import Counter
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
import base64
import matplotlib.pyplot as plt
from datetime import datetime
import pytz


# Initialize the connection to Firebase
firebase_url = 'https://elephant-994c6-default-rtdb.firebaseio.com/'  # Replace with your Firebase URL
FBconn = requests.Session()

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Global Variables
uploaded_data = None
filtered_df = pd.DataFrame()  # Initialize filtered_df as an empty DataFrame
chatbot = None  # Define chatbot globally

# Define action weights for calculating the best performer
action_weights = {
    'create_document': 10,
    'edit_document': 5,
    'view_document': 1
}

# Define the buttons globally
chat_button = widgets.Button(
    description='Chat',
    disabled=False,
    button_style='info',
    tooltip='Click to chat with the assistant',
    icon='comments'
)

download_pdf_button = widgets.Button(
    description='Download as PDF',
    disabled=False,
    button_style='success',
    tooltip='Click to download filtered results as PDF',
    icon='download'
)

show_graphs_button = widgets.Button(
    description='Show Graphs',
    disabled=False,
    button_style='info',
    tooltip='Click to show graphs for filtered results',
    icon='bar-chart'
)

back_to_upload_button = widgets.Button(
    description='Back to Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to go back to upload page',
    icon='arrow-left'
)

# Tailwind CSS to style and center elements
tailwind_css = """
<link href="https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css" rel="stylesheet">
<style>
    body {
        background-color: #f4f4f9;
        font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
        color: #333;
    }
    .centered {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        flex-direction: column;
        text-align: center;
    }
    .form-container {
        background-color: #ffffff;
        padding: 20px;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        max-width: 350px;
        width: 100%;
        margin: auto;
    }
    .page-title {
        color: #1e3a8a;
        font-size: 2rem;
        font-weight: bold;
        margin-bottom: 1rem;
    }
    .form-title {
        color: #1e3a8a;
        font-size: 1.5rem;
        font-weight: bold;
        margin-bottom: 1rem;
    }
    .form-label {
        color: #333;
        font-weight: bold;
        margin-bottom: 0.5rem;
        display: block;
    }
    .form-input {
        width: 100%;
        padding: 10px;
        margin-bottom: 1rem;
        border: 1px solid #ccc;
        border-radius: 4px;
        box-sizing: border-box;
    }
    .form-button {
        width: 100%;
        padding: 10px;
        background-color: #1e7e34;
        color: white;
        border: none;
        border-radius: 4px;
        cursor: pointer;
        font-weight: bold;
        text-transform: uppercase;
    }
    .form-button:hover {
        background-color: #218838;
    }
    .notification {
        color: red;
        font-weight: bold;
    }
</style>
"""

# Inject Tailwind CSS
display(HTML(tailwind_css))

# Function to fetch data from Firebase
def fetch_data_from_firebase():
    url = f'{firebase_url}.json'
    result = FBconn.get(url)
    if result.status_code == 200:
        return result.json()
    else:
        return {}

# Function to calculate user scores based on weighted actions
def calculate_user_scores(data):
    user_scores = Counter()

    for entry in data:
        user = entry.get('User')
        description = entry.get('Description')

        # Assign scores based on action type or description
        if 'Create document' in description or 'Create' in description:
            user_scores[user] += 5
        elif 'Insert' in description or 'Add' in description:
            user_scores[user] += 3
        elif 'Update' in description or 'Modify' in description:
            user_scores[user] += 2
        elif 'Delete' in description:
            user_scores[user] -= 1
        else:
            user_scores[user] += 1

    return user_scores


# Function to initialize the chatbot based on data
def initialize_chatbot():
    global chatbot
    data = fetch_data_from_firebase()

    if isinstance(data, list):
        user_scores = calculate_user_scores(data)
        if user_scores:
            best_performer = max(user_scores, key=user_scores.get)
            best_performer_message = f"The best performer is {best_performer} with a score of {user_scores[best_performer]}."
        else:
            best_performer_message = "No actions recorded yet, so there's no best performer."
        users = Counter([item.get('User') for item in data])
        documents = Counter([item.get('Document') for item in data if item.get('Document')])
        tabs = Counter([item.get('Tab') for item in data if item.get('Tab')])
        descriptions = Counter([item.get('Description') for item in data if item.get('Description')])
    else:
        print("Unexpected data format: Data is not a list.")
        return

    total_actions = len(data)
    unique_users = len(users)
    unique_documents = len(documents)
    unique_tabs = len(tabs)

    patterns = [
        (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Welcome to the project management assistant.']),
        (r'how are you?', ["I'm functioning well, thank you!", "I'm operational and ready to assist with your project management."]),
        (r'what is your name?', ["I'm the Project Management Assistant.", "You can call me the Assistant."]),
        (r'How many total actions are recorded?', [f"There are {total_actions} total actions recorded in the database."]),
        (r'Who is the most active user?', [f"The most active user is {users.most_common(1)[0][0]} with {users.most_common(1)[0][1]} actions."]),
        (r'How many unique users are there?', [f"There are {unique_users} unique users in the database."]),
        (r'What is the most frequently accessed document?', [f"The most frequently accessed document is '{documents.most_common(1)[0][0]}' with {documents.most_common(1)[0][1]} accesses."]),
        (r'How many different documents were accessed?', [f"There were {unique_documents} different documents accessed."]),
        (r'What is the most common tab used?', [f"The most commonly used tab is '{tabs.most_common(1)[0][0]}' with {tabs.most_common(1)[0][1]} uses."]),
        (r'What is the most frequent action description?', [f"The most frequent action description is '{descriptions.most_common(1)[0][0]}' occurring {descriptions.most_common(1)[0][1]} times."]),
        (r'What are the top 3 most active users?', [
            f"The top 3 most active users are: 1. {users.most_common(3)[0][0]} ({users.most_common(3)[0][1]} actions), "
            f"2. {users.most_common(3)[1][0]} ({users.most_common(3)[1][1]} actions), "
            f"3. {users.most_common(3)[2][0]} ({users.most_common(3)[2][1]} actions)."]),
        (r'What is the least used tab?', [f"The least used tab is '{tabs.most_common()[-1][0]}' with only {tabs.most_common()[-1][1]} uses."]),
        (r'How many different types of actions \(descriptions\) are there?', [f"There are {len(descriptions)} different types of actions (unique descriptions) in the database."]),
        (r'who is the best performer\??', [best_performer_message]),
        (r'exit|bye|goodbye', ["Thank you for using the Project Management Assistant. Goodbye!", "Farewell! Don't hesitate to return if you need more assistance with your project."]),
    ]

    chatbot = Chat(patterns, reflections)

# Function to display the login page
def display_login_page():
    clear_output()

    username = widgets.Text(
        value='',
        placeholder='Enter your username',
        layout=widgets.Layout(width='30%', padding='5px', margin='0 0 1rem 0', border='1px solid #ccc', border_radius='2px')
    )
    password = widgets.Password(
        value='',
        placeholder='Enter your password',
        layout=widgets.Layout(width='30%', padding='5px', margin='0 0 1rem 0', border='1px solid #ccc', border_radius='2px')
    )
    login_button = widgets.Button(
        description='Login',
        disabled=False,
        button_style='',
        tooltip='Click to login',
        icon='check',
        layout=widgets.Layout(width='20%', padding='5px', margin='5px 0'),
        style=dict(button_color='#21ab0f', font_weight='bold', text_transform='uppercase')
    )
    login_message = widgets.Label()

    def on_login_button_clicked(b):
        if username.value == 'admin' and password.value == 'admin':
            clear_output()
            display_upload_page()
        else:
            login_message.value = 'Invalid username or password. Please try again.'

    login_button.on_click(on_login_button_clicked)

    display(widgets.VBox([
        widgets.HTML("<h1 class='page-title' style='color:#1e3a8a;'>Elephant Application</h1>"),
        widgets.HTML("<h2 class='form-title'>Login</h2>"),
        widgets.HTML("<div class='form-container'>"),
        widgets.HTML("<label class='form-label'>Username</label>"),
        username,
        widgets.HTML("<label class='form-label'>Password</label>"),
        password,
        login_button,
        login_message,
        widgets.HTML("</div>")
    ], layout=widgets.Layout(align_items='center', justify_content='center', height='100vh')))

# Define the upload page elements
def display_upload_page():
    upload_button = widgets.FileUpload(
        accept='.json',
        multiple=False
    )
    upload_button_description = widgets.Label(value='Upload a JSON file to replace the old one in the database.')
    proceed_button = widgets.Button(
        description='Proceed to Search Page',
        disabled=False,
        button_style='success',
        tooltip='Click to proceed to the search page',
        icon='arrow-right'
    )
    back_button = widgets.Button(
        description='Back to Login',
        disabled=False,
        button_style='warning',
        tooltip='Click to go back to login page',
        icon='arrow-left'
    )
    upload_message = widgets.Label()

    def on_upload_button_change(change):
        global uploaded_data
        for filename, file_info in upload_button.value.items():
            if not filename.endswith('.json'):
                upload_message.value = 'Error: Wrong file format. Please upload a JSON file.'
                return
            uploaded_data = json.loads(file_info['content'].decode('utf-8'))
            url = f'{firebase_url}.json'
            response = FBconn.put(url, json=uploaded_data)
            if response.status_code == 200:
                upload_message.value = 'File uploaded successfully.'
            else:
                upload_message.value = 'File upload failed.'

    upload_button.observe(on_upload_button_change, names='value')

    def on_proceed_button_clicked(b):
        if uploaded_data is not None:
            clear_output()
            display_search_page()
        else:
            upload_message.value = 'Please upload a JSON file before proceeding.'

    proceed_button.on_click(on_proceed_button_clicked)

    def on_back_button_clicked(b):
        display_login_page()

    back_button.on_click(on_back_button_clicked)

    clear_output()
    display(widgets.VBox([
        widgets.HTML("<h1 style='color:#1e3a8a; font-size:2rem; font-weight:bold; margin-bottom:1rem;'>Upload Page</h1>"),
        upload_button_description,
        upload_button,
        upload_message,
        widgets.HBox([proceed_button, back_button], layout=widgets.Layout(justify_content='center', gap='10px'))
    ], layout=widgets.Layout(align_items='center', justify_content='center', height='100vh')))

# Define the search page elements
def display_search_page():
    global from_date, to_date, tab_selector, user_selector, description_input
    global uploaded_data, filtered_df

    no_results_message = widgets.HTML(value="", layout=widgets.Layout(visibility='hidden', color='red', margin='10px 0'))

    if uploaded_data is None:
        data = fetch_data_from_firebase()
    else:
        data = uploaded_data

    df = pd.DataFrame(data)

    df['Time'] = pd.to_datetime(df['Time'])

    unique_tabs = df['Tab'].unique().tolist()
    unique_tabs.insert(0, "All")

    unique_users = df['User'].unique().tolist()
    unique_users.insert(0, "All")

    from_date = widgets.DatePicker(description='From', disabled=False)
    to_date = widgets.DatePicker(description='To', disabled=False)

    tab_selector = widgets.Dropdown(options=unique_tabs, value="All", description='Tab:', disabled=False)
    user_selector = widgets.Dropdown(options=unique_users, value="All", description='User:', disabled=False)
    description_input = widgets.Text(value='', placeholder='Enter description to search', description='Description:', disabled=False)

    def filter_data(change):
        global filtered_df
        no_results_message.layout.visibility = 'hidden'
        if from_date.value and to_date.value:
            if from_date.value > to_date.value:
                no_results_message.value = 'Error: "From" date must be earlier than "To" date.'
                no_results_message.layout.visibility = 'visible'
                return
            start_date = pd.to_datetime(from_date.value)
            end_date = pd.to_datetime(to_date.value)
            selected_tab = tab_selector.value
            selected_user = user_selector.value
            description_text = description_input.value.lower()

            filtered_df = df[(df['Time'] >= start_date) & (df['Time'] <= end_date)]
            if selected_tab != "All":
                filtered_df = filtered_df[filtered_df['Tab'] == selected_tab]
            if selected_user != "All":
                filtered_df = filtered_df[filtered_df['User'] == selected_user]
            if description_text:
                filtered_df = filtered_df[filtered_df['Description'].str.lower().str.contains(description_text)]

            if filtered_df.empty:
                no_results_message.value = 'No results found for the specified search criteria.'
                no_results_message.layout.visibility = 'visible'

            clear_output(wait=True)
            display_search_page_elements(no_results_message)
            display_table(filtered_df)

    def on_chat_button_clicked(b):
        display_chatbot_page()

    chat_button.on_click(on_chat_button_clicked)

    # Handler for back to upload
    def on_back_to_upload_button_clicked(b):
        display_upload_page()

    back_to_upload_button.on_click(on_back_to_upload_button_clicked)

    # Handler for showing graphs
    def on_show_graphs_button_clicked(b):
        if not filtered_df.empty:
            clear_output(wait=True)
            display_search_page_elements(no_results_message)
            display_table(filtered_df)
            display_graphs(filtered_df)

    show_graphs_button.on_click(on_show_graphs_button_clicked)

    # Handler for downloading PDF
    def on_download_pdf_button_clicked(b):
        if not filtered_df.empty:
            generate_pdf(filtered_df)

    download_pdf_button.on_click(on_download_pdf_button_clicked)

    from_date.observe(filter_data, names='value')
    to_date.observe(filter_data, names='value')
    tab_selector.observe(filter_data, names='value')
    user_selector.observe(filter_data, names='value')
    description_input.observe(filter_data, names='value')

    display_search_page_elements(no_results_message)

# Function to display search page elements
def display_search_page_elements(no_results_message):
    display(widgets.VBox([
        widgets.HTML("<h1 style='color:#1e3a8a; font-size:2rem; font-weight:bold; margin-bottom:1rem;'>Search Page</h1>"),
        from_date,
        to_date,
        tab_selector,
        user_selector,
        description_input,
        no_results_message,
        widgets.HBox([download_pdf_button, show_graphs_button, chat_button], layout=widgets.Layout(justify_content='center', gap='10px')),
        back_to_upload_button
    ], layout=widgets.Layout(align_items='center', justify_content='center', height='100vh')))

# Function to display the table
def display_table(dataframe):
    display(widgets.HTML("<div class='table-section'>"))
    display_paginated(dataframe)
    display(widgets.HTML("</div>"))

# Function to display paginated data
def display_paginated(dataframe, rows_per_page=10):
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    def display_page(page):
        clear_output(wait=True)
        display_search_page_elements(widgets.HTML(value=""))
        start = page * rows_per_page
        end = start + rows_per_page
        table_html = dataframe.iloc[start:end].to_html(classes='table table-striped table-bordered', index=False)
        display(HTML(f'<style>.dataframe td {{ white-space: nowrap; overflow: hidden; text-overflow: ellipsis; max-width: 400px; }}</style>{table_html}'))
        display(page_slider)

    n_pages = len(dataframe) // rows_per_page + 1
    page_slider = widgets.IntSlider(min=0, max=n_pages - 1, step=1, description='Page')

    def on_page_change(change):
        display_page(change['new'])

    page_slider.observe(on_page_change, names='value')
    display(page_slider)
    display_page(0)

# Function to display graphs
def display_graphs(dataframe):
    plt.figure(figsize=(10, 6))

    # Example graph: Number of entries per User
    user_counts = dataframe['User'].value_counts()
    user_counts.plot(kind='bar')
    plt.title('Number of Entries per User')
    plt.xlabel('User')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example graph: Number of entries per Tab
    tab_counts = dataframe['Tab'].value_counts()
    tab_counts.plot(kind='bar', color='orange')
    plt.title('Number of Entries per Tab')
    plt.xlabel('Tab')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example graph: Number of entries per Document
    doc_counts = dataframe['Document'].value_counts()
    doc_counts.plot(kind='bar', color='green')
    plt.title('Number of Entries per Document')
    plt.xlabel('Document')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example graph: Entries over Time
    dataframe['Time'].dt.to_period('M').value_counts().sort_index().plot(kind='line', marker='o')
    plt.title('Entries Over Time')
    plt.xlabel('Time')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example pie chart: Distribution of entries per User
    user_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title('Distribution of Entries per User')
    plt.ylabel('')
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example pie chart: Distribution of entries per Tab
    tab_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
    plt.title('Distribution of Entries per Tab')
    plt.ylabel('')
    plt.show()

# Function to generate PDF from DataFrame
def generate_pdf(dataframe, filename='filtered_data.pdf'):
    doc = SimpleDocTemplate(filename, pagesize=letter)
    elements = []
    data = [dataframe.columns.tolist()] + dataframe.values.tolist()
    table = Table(data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    doc.build(elements)

    # Trigger the file download immediately
    with open(filename, 'rb') as f:
        pdf_data = f.read()
    b64_pdf = base64.b64encode(pdf_data).decode()
    js = f"""
        var link = document.createElement('a');
        link.href = 'data:application/pdf;base64,{b64_pdf}';
        link.download = '{filename}';
        link.click();
    """
    display(HTML(f'<script>{js}</script>'))

from datetime import datetime
import pytz

# Define the chatbot page elements and logic with enhanced design
def display_chatbot_page():
    clear_output()

    output_area = widgets.Output(
        layout={
            'border': '1px solid #ccc',
            'height': '300px',  # Adjusted height for better visibility
            'overflow': 'auto',
            'padding': '10px',
            'border-radius': '5px',
            'width': '50%'  # Adjust width to suit your design preference
        }
    )

    user_input = widgets.Text(
        value='',
        placeholder='Type your message here...',
        description='You:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

    submit_button = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='primary',
        tooltip='Click to submit your message',
        icon='check'
    )

    back_button = widgets.Button(
        description='Back to Search',
        disabled=False,
        button_style='warning',
        tooltip='Click to go back to the search page',
        icon='arrow-left'
    )

    def on_submit_enhanced(b=None):
        with output_area:
            user_message = user_input.value
            user_input.value = ''

            # Set the timezone for Israel and format the timestamp
            tz = pytz.timezone('Asia/Jerusalem')
            time_stamp = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

            # Display user's message with message bubble style
            display(HTML(f"""
            <div style='text-align:right; margin: 5px;'>
                <span style='display:inline-block; padding: 10px; background-color: #31d5fa; color: black; border-radius: 10px; max-width: 80%; word-wrap: break-word;'>
                    <strong>You [{time_stamp}]:</strong><br>{user_message}
                </span>
            </div>
            """))

            response = chatbot.respond(user_message)

            # Add a fallback response if the chatbot returns None or an empty string
            if not response:
                response = "I'm sorry, I didn't quite understand that. Could you please rephrase?"

            # Display chatbot's response with message bubble style
            display(HTML(f"""
            <div style='text-align:left; margin: 5px;'>
                <span style='display:inline-block; padding: 10px; background-color: #5bdf8b; color: black; border-radius: 10px; max-width: 80%; word-wrap: break-word;'>
                    <strong>Chatbot [{time_stamp}]:</strong><br>{response}
                </span>
            </div>
            """))

    # Link the submit button to the enhanced submit function
    submit_button.on_click(on_submit_enhanced)

    # Handle "Enter" key to submit the form
    user_input.on_submit(on_submit_enhanced)

    back_button.on_click(lambda b: display_search_page())  # Back to search page

    display(widgets.VBox([output_area, user_input, submit_button, back_button]))


# Initialize chatbot with data from Firebase
initialize_chatbot()

# Start the application by displaying the login page
display_login_page()
